In [1]:
import sys
print(sys.version)

3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]


In [2]:
!pip install tensorflow

In [3]:
!pip install optuna

In [4]:
!pip install scikit-learn

In [5]:
!pip install numpy==1.22


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 65.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
chex 0.1.85 requires numpy>=1.24.1, but you have numpy 1.22.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.3 which is incompatible.
cudf 23.

In [6]:
!pip install tensorflow-addons


In [7]:
#necessary imports

import numpy as np
import struct
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape, Dense, Dropout, BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler

import optuna

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [8]:
def read_mnist_images(filename):
    with open(filename, 'rb') as file:
        magic, num_images, rows, cols = struct.unpack(">IIII", file.read(16)) #read the first 16 bytes that contain the metadata
        print(f"Number of images: {num_images}")
        print(f"Number of rows: {rows}")
        print(f"Number of columns: {cols}")

        image_data = np.fromfile(file, dtype=np.uint8) #read the rest of the file as unsigned bytes
        images = image_data.reshape(num_images, rows, cols) #reshape the data into 3D array 
    return images, num_images, rows, cols
filename = 'dataset.dat' #the path to the dataset
images, num_images, rows, cols = read_mnist_images(filename) #read the images
images = images.astype(np.float32) / 255.0
images = np.expand_dims(images, axis=-1)
X_train, X_test = train_test_split(images, test_size=0.2)   #split the data into train and test sets with 80% and 20% respectively

Number of images: 60000
Number of rows: 28
Number of columns: 28


In [9]:
from tensorflow.keras.layers import Cropping2D, ZeroPadding2D


In [10]:

def scheduler(epoch, lr):   #function to reduce the learning rate after 10 epochs used in the LearningRateScheduler callback
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)


# Usage
def create_autoencoder(trial, input_dim, bottleneck_size):  # Function to create the autoencoder model
    n_conv_layers = trial.suggest_int('n_conv_layers', 2, 5)    # Example range for conv layers
    activation = trial.suggest_categorical('activation', ['relu', 'sigmoid', 'tanh']) 
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.1) # Example range for dropout rate

    input_img = Input(shape=input_dim)
    x = input_img
    
    # Encoding part
    filters_list = [] # List of filters for each layer
    kernel_list = [] # List of kernel sizes for each layer
    for i in range(n_conv_layers): # Loop over the layers to get the number of filters and kernel sizes per layer
        filters = trial.suggest_int(f'filters_layer_{i}', 16, 64)  # Adjusted filter range
        filters_list.append(filters)
        kernel_size_str = trial.suggest_categorical(f'kernel_size_layer_{i}', ['3x3','5x5'])
        kernel_size = tuple(map(int, kernel_size_str.split('x')))
        kernel_list.append(kernel_size)
    filters_list.sort(reverse=True)
    to_pool = []   # List of booleans to indicate whether to pool or not
    #We will pool until we reach 3x3
    for i in range(n_conv_layers):
        x = Conv2D(filters_list[i], kernel_list[i], activation=activation, padding='same')(x)   # Convolutional layer
        x = BatchNormalization()(x) # Batch normalization
        if x.shape[1] != 3 : # If the shape is 3x3, we don't pool
            x = MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid')(x) # Pooling layer to reduce the dimensions by half 
            to_pool.append(True) # Add True to the list to indicate that we pooled
        else : 
            to_pool.append(False) # Add False to the list to indicate that we didn't pool
        x = Dropout(dropout_rate)(x) # Dropout layer to prevent overfitting

    # Bottleneck
    x = Dense(bottleneck_size, activation=activation)(x) # Dense layer to reduce the dimensions to the bottleneck size - latent space
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)

    # Decoding part
    for i in range(n_conv_layers): # Loop over the layers to get the number of filters and kernel sizes per layer
        if to_pool[n_conv_layers - i - 1]: # If we pooled in the encoding part, we upsample
            if x.shape[1] == 3:  # Custom handling for 3x3 to 7x7
                x = UpSampling2D(size=(2, 2))(x)  # Upsample to 6x6
                x = ZeroPadding2D(padding=((0, 1), (0, 1)))(x)  # Pad to 7x7
            else:
                x = UpSampling2D(size=(2, 2))(x)  # Standard upsampling
        x = Conv2D(filters_list[n_conv_layers - i - 1], kernel_list[n_conv_layers - i - 1], activation=activation, padding='same')(x) # Convolutional layer
        x = BatchNormalization()(x)
        x = Dropout(dropout_rate)(x)
    # Final Conv2D layer to match the input shape

    autoencoder = Model(input_img, x)
    return autoencoder




def objective(trial):

    bottleneck_size = trial.suggest_int('bottleneck_size', 10, 40) # Example range for N
    autoencoder = create_autoencoder(trial, (rows, cols, 1), bottleneck_size)
    autoencoder.summary()
    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'sgd', 'rmsprop', 'adamw'])
    lr = trial.suggest_float('lr', 1e-3, 1e-1, log=True)
    if optimizer_name == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer_name == 'sgd':
        momentum = trial.suggest_float('momentum', 0.0, 0.9)
        optimizer = tf.keras.optimizers.SGD(learning_rate=lr, momentum=momentum)
    elif optimizer_name == 'rmsprop':
        rho = trial.suggest_float('rho', 0.8, 0.99)
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr, rho=rho)
    elif optimizer_name == 'adamw':
    # AdamW is not a part of standard Keras optimizers, so it might require a custom implementation or a library like tf-addons
        weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-2, log=True)
        optimizer = tfa.optimizers.AdamW(learning_rate=lr, weight_decay=weight_decay)
    autoencoder.compile(optimizer=optimizer, loss='mean_squared_error') # Compile the model with the optimizer and the loss function

    batch_size = trial.suggest_int('batch_size', 64, 512, log=True)
    epochs = trial.suggest_int('epochs', 5, 15)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True) # Early stopping callback to stop training if the validation loss doesn't improve for 5 epochs
    lr_scheduler = LearningRateScheduler(scheduler) # Learning rate scheduler callback to reduce the learning rate after 10 epochs
    autoencoder.fit(X_train, X_train, epochs=epochs, batch_size=batch_size, shuffle=True, 
                    validation_data=(X_test, X_test), callbacks=[early_stopping, lr_scheduler])

    test_loss = autoencoder.evaluate(X_test, X_test, verbose=0)
    return test_loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Best trial:")
trial = study.best_trial
print("    Value: ", trial.value)
print("    Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2024-01-06 18:46:09,195] A new study created in memory with name: no-name-44a07244-4a72-45fd-8475-55dadf3d889c


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 60)        1560      
                                                                 
 batch_normalization (Batch  (None, 28, 28, 60)        240       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 60)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 14, 14, 60)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 59)        88559 

2024-01-06 18:46:14.540158: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


281/281 [==============================] - 25s 47ms/step - loss: 0.1071 - val_loss: 0.0809 - lr: 0.0358
Epoch 2/6
281/281 [==============================] - 12s 41ms/step - loss: 0.0673 - val_loss: 0.0781 - lr: 0.0358
Epoch 3/6
281/281 [==============================] - 12s 42ms/step - loss: 0.0668 - val_loss: 0.0683 - lr: 0.0358
Epoch 4/6
281/281 [==============================] - 12s 42ms/step - loss: 0.0687 - val_loss: 0.0693 - lr: 0.0358
Epoch 5/6
281/281 [==============================] - 12s 42ms/step - loss: 0.0706 - val_loss: 0.0880 - lr: 0.0358
Epoch 6/6
281/281 [==============================] - 12s 43ms/step - loss: 0.0722 - val_loss: 0.0712 - lr: 0.0358


[I 2024-01-06 18:47:37,685] Trial 0 finished with value: 0.07118882238864899 and parameters: {'bottleneck_size': 32, 'n_conv_layers': 5, 'activation': 'tanh', 'dropout_rate': 0.019209991404743856, 'filters_layer_0': 59, 'kernel_size_layer_0': '5x5', 'filters_layer_1': 60, 'kernel_size_layer_1': '5x5', 'filters_layer_2': 34, 'kernel_size_layer_2': '5x5', 'filters_layer_3': 56, 'kernel_size_layer_3': '5x5', 'filters_layer_4': 32, 'kernel_size_layer_4': '3x3', 'optimizer': 'adam', 'lr': 0.03580161053135974, 'batch_size': 171, 'epochs': 6}. Best is trial 0 with value: 0.07118882238864899.


...

Model: "model_49"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_50 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_268 (Conv2D)         (None, 28, 28, 59)        1534      
                                                                 
 batch_normalization_317 (B  (None, 28, 28, 59)        236       
 atchNormalization)                                              
                                                                 
 max_pooling2d_118 (MaxPool  (None, 14, 14, 59)        0         
 ing2D)                                                          
                                                                 
 dropout_317 (Dropout)       (None, 14, 14, 59)        0         
                                                                 
 conv2d_269 (Conv2D)         (None, 14, 14, 41)        605

2024-01-06 20:14:48.856128: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_49/dropout_317/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


388/388 [==============================] - 14s 25ms/step - loss: 0.0320 - val_loss: 0.0436 - lr: 0.0733
Epoch 2/9
388/388 [==============================] - 9s 23ms/step - loss: 0.0080 - val_loss: 0.0066 - lr: 0.0733
Epoch 3/9
388/388 [==============================] - 9s 23ms/step - loss: 0.0066 - val_loss: 0.0048 - lr: 0.0733
Epoch 4/9
388/388 [==============================] - 9s 23ms/step - loss: 0.0059 - val_loss: 0.0067 - lr: 0.0733
Epoch 5/9
388/388 [==============================] - 9s 23ms/step - loss: 0.0055 - val_loss: 0.0039 - lr: 0.0733
Epoch 6/9
388/388 [==============================] - 9s 23ms/step - loss: 0.0051 - val_loss: 0.0027 - lr: 0.0733
Epoch 7/9
388/388 [==============================] - 9s 23ms/step - loss: 0.0049 - val_loss: 0.0028 - lr: 0.0733
Epoch 8/9
388/388 [==============================] - 9s 23ms/step - loss: 0.0047 - val_loss: 0.0026 - lr: 0.0733
Epoch 9/9
388/388 [==============================] - 9s 23ms/step - loss: 0.0046 - val_loss: 0.0020 - lr:

[I 2024-01-06 20:16:13,904] Trial 49 finished with value: 0.0020103021524846554 and parameters: {'bottleneck_size': 16, 'n_conv_layers': 2, 'activation': 'relu', 'dropout_rate': 0.0171533939020817, 'filters_layer_0': 41, 'kernel_size_layer_0': '5x5', 'filters_layer_1': 59, 'kernel_size_layer_1': '5x5', 'optimizer': 'adam', 'lr': 0.07332369385755381, 'batch_size': 124, 'epochs': 9}. Best is trial 34 with value: 0.0015369041357189417.


Study statistics: 
  Number of finished trials:  50
  Best trial:
    Value:  0.0015369041357189417
    Params: 
    bottleneck_size: 15
    n_conv_layers: 2
    activation: relu
    dropout_rate: 0.00044795018064423126
    filters_layer_0: 45
    kernel_size_layer_0: 5x5
    filters_layer_1: 46
    kernel_size_layer_1: 5x5
    optimizer: adam
    lr: 0.014125762989878794
    batch_size: 223
    epochs: 15
